In [68]:
import pandas as pd
import os
from os.path import join  
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [2]:
prelim_dir = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2/Midstream/PRELIM_v1.6.xlsm'

In [3]:
sp_dir = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2'

In [4]:

field_names_path = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/ClimateTRACE Phase 4/csv/db_uploads/field_name_df.csv'

field_names_df = pd.read_csv(field_names_path)


### Extract Assay Information

In [5]:
def extract_assay_info(file):
    
    df = pd.read_excel(file,sheet_name='Assay Inventory', na_values=['NA'], usecols="B:E",skiprows=7, names=['Country', 'Selected Assay', 'Units', 'Full Crude'])
    
    country_list = []
    region_list = []
    assay_names = []
    assay_numbers = []
    full_crude_vol_flows = []
    full_crude_sulphurs = []
    full_crude_apis = []

    for i in df.index:
        
        country_region = df.at[i,'Country']
        if pd.notnull(country_region):
            
            cr_list = country_region.split('-')
            
            if len(cr_list) > 1:
                
                country = cr_list[0]
                region = cr_list[1]
                
                country_list.append(country)
                region_list.append(region)
            
            else:
                country_list.append(country_region)
                region_list.append(np.nan)
                
            assay_names.append(df.at[i,'Selected Assay'])
            assay_numbers.append(df.at[i+1,'Units'])

            #get vol flow
            full_crude_vol_flows.append(df.at[i+3,'Full Crude'])

            #get sulpher
            full_crude_sulphurs.append(df.at[i+6,'Full Crude'])

            #get api
            full_crude_apis.append(df.at[i+8,'Full Crude'])



    #df_assay = pd.DataFrame(list(zip(country_list, region_list, assay_names, assay_numbers, units_flow, full_crude_vol_flows, units_sulphur, full_crude_sulphurs,units_api,full_crude_apis)), columns=['Country','Region','Assay Name','Assay Number','Throughput Unit','Throughput','Sulphur Unit','Sulphur','API Unit','API'])
    df_assay = pd.DataFrame(list(zip(country_list, region_list, assay_names, assay_numbers, full_crude_vol_flows, full_crude_sulphurs,full_crude_apis)), columns=['Country','Region','Assay Name','Assay Number','Throughput','Sulphur','API'])
    
    return df_assay
        

In [6]:
assay_df = extract_assay_info(prelim_dir)

#### Separate Assay Name from Year

In [7]:
import re
split_assay_year = [int(re.findall(r'\b\d+\b',x)[0]) if len(re.findall(r'\b\d+\b',x))>0 else np.nan for x in assay_df['Assay Name']]
assay_df['Year'] = [int(x) if x > 1000 else np.nan for x in split_assay_year]
assay_df['Year'] = assay_df['Year'].astype('Int64')
assay_df['Assay Name NN'] = [" ".join(x.split()[:-1]) if x.split()[-1].isnumeric() else x for x in assay_df['Assay Name']]

In [67]:
assay_df.head(2)

,Country,Region,Assay Name,Assay Number,Throughput,Sulphur,API,Year,Assay Name NN
0,Nigeria,NaN,Abo_Solomon 2010,11,100032.667139,0.147911,36.396513,2010,Abo_Solomon
1,Canada,Alberta,Access Western Blend_Crude Monitor,NaN,99851.088615,4.196450,22.640000,<NA>,Access Western Blend_Crude Monitor


### Match Assay to Field Name

Will capture when multiple assays match a single field, will also capture when multiple fields match the same assay

In [13]:
rmi_name_list = []
assay_matches = []

for i in field_names_df.index:
    FN = field_names_df.at[i,'Field_name']
    assay_names = assay_df['Assay Name NN']
    match_list = [x for x in assay_names if FN in x]
    
    #If more than one assay matches
    if len(match_list) > 0:
        
        rmi_name_list.append(FN)
        assay_matches.append(match_list)
        
    else:
        
        
        
    df_match = pd.DataFrame(list(zip(rmi_name_list, assay_matches)), columns=['RMI Name','Assay Match'])


### Unpack the list of assay names

In [14]:
rmi_name = []
assay_match_name =[]
for i in df_match.index:
    for match in df_match.at[i,'Assay Match']:
        rmi_name.append(df_match.at[i,'RMI Name'])
        assay_match_name.append(match)


df_match_unpack = pd.DataFrame(list(zip(rmi_name, assay_match_name)), columns = ['RMI Name', 'Assay Name NN']) 

#### Merge the Assay information into the RMI Name

In [16]:
df_full_match = df_match_unpack.merge(assay_df, how='left', on='Assay Name NN')

### Merge with Field Info From Runs

In [55]:
opgee_output_path = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/ClimateTRACE Phase 4/csv/db_exports/trace_main_outputs.csv'
opgee_outpus = pd.read_csv(opgee_output_path)
opgee_outputs_2 = opgee_outpus.loc[:,['Field_name','Region','API_gravity']] #'Gas_composition_H2S'


/Users/atecza/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (225) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### How do we want to merge? Do we want to bring in API for just one year? 

In [56]:
opgee_outputs_2.rename(columns={'Year':'RMI Year','Region':'RMI Region','API_gravity':'RMI API'}, inplace=True)

In [57]:
opgee_outputs_group = opgee_outputs_2.groupby('Field_name').first().reset_index()

In [58]:
opgee_assay_full = df_full_match.merge(opgee_outputs_group, how = 'left', left_on ='RMI Name', right_on='Field_name')

In [66]:
#[x for x in list(opgee_assay_full.columns) if 'Year' in x]
opgee_assay_full.head(2)

,RMI Name,Assay Name NN,Country,Region,Assay Name,Assay Number,Throughput,Sulphur,API,Year,Field_name,RMI Region,RMI API,API Difference
0,Agbami,Nigeria Agbami_Chevron,Nigeria,NaN,Nigeria Agbami_Chevron,32,95429.629390,0.081672,47.876196,<NA>,Agbami,NaN,47.0,0.876196
1,Agbami,Nigeria Agbami_Statoil,Nigeria,NaN,Nigeria Agbami_Statoil,55,95915.130925,0.072768,48.032857,<NA>,Agbami,NaN,47.0,1.032857


In [60]:
opgee_assay_full['API Difference'] = abs(opgee_assay_full['API'] - opgee_assay_full['RMI API'])

In [61]:
series_list = []
df_group = opgee_assay_full.groupby('RMI Name')
for name_of_group, contents_of_group in df_group:
    df1 = df_group.get_group(name_of_group)
    if df1['Year'].isnull().any():
        series_list.append(df1.loc[df1['API Difference'].idxmin()])
    
    else:
        series_list.append(df1.loc[df1['Year'].idxmax()])

df_g = pd.concat(series_list, axis=1).T

In [65]:
print(opgee_assay_full.groupby('RMI Name').ngroups)
print(len(df_g))

83
83


### Old Code/Sandbox

In [315]:
assay_df['API Diff Normalized'] = abs((assay_df['API'] - assay_df['API'].mean())/(assay_df['API'].std()))
#assay_df['API Normalized'] = abs((assay_df['API'] - assay_df['API'].mean())/(assay_df['API'].std()))

In [109]:
def closest_val(lst, K):
    closest_val_list = []
    dif_list = [abs(x-K) for x in lst]
    s = min(dif_list)

    i_list = [i for i, x in enumerate(dif_list) if x == s]

    for index in i_list:
        closest_val_list.append(lst[index])
    
    return closest_val_list